## Multiclass

In [ ]:
# Use multiple GPUs or not

use_multi_gpu = False

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

import tensorflow as tf
import pandas as pd
import numpy as np

attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

features = ['Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']
for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')


batch_size = 64
img_height = 128
img_width = 128

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)



2023-05-28 20:04:59.116076: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-28 20:05:00.293889: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 20:05:05.232685: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [1]:
# Distribute load over all GPU

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

if use_multi_gpu:
    def set_gpus(*gpu_indices):
        # Get list of GPUs
        gpus = tf.config.experimental.list_physical_devices('GPU')
        print("Num GPUs Available: ", len(gpus))

        if gpus:
            visible_gpus = [gpus[i] for i in gpu_indices]
            try:
                # Only use specified GPUs and ignore the others
                tf.config.experimental.set_visible_devices(visible_gpus, 'GPU')
                for gpu in visible_gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                print("Using GPUs: ", [gpu.name for gpu in visible_gpus])
            except RuntimeError as e:
                print(e)
        return visible_gpus

    # Call the function with the indices of the GPUs you want to use
    visible_gpus = set_gpus(1, 2, 3, 4)  # Use the second, third, fourth, and fifth GPU

    # Extract names of the GPUs being used
    gpu_names = ['/device:GPU:' + gpu.name.split(':')[-1] for gpu in visible_gpus]

    strategy = tf.distribute.MirroredStrategy(devices=gpu_names)

2023-06-01 20:21:51.052491: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 20:21:51.128932: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 20:21:53.079088: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  5
Using GPUs:  ['/physical_device:GPU:1', '/physical_device:GPU:2', '/physical_device:GPU:3', '/physical_device:GPU:4']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


2023-06-01 20:22:00.401612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 35839 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:1b:00.0, compute capability: 8.6
2023-06-01 20:22:00.402509: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 40959 MB memory:  -> device: 2, name: NVIDIA RTX A6000, pci bus id: 0000:88:00.0, compute capability: 8.6
2023-06-01 20:22:00.403121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 40959 MB memory:  -> device: 3, name: NVIDIA RTX A6000, pci bus id: 0000:89:00.0, compute capability: 8.6
2023-06-01 20:22:00.403806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 40959 MB memory:  -> device: 4, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, 

### Custom model 1

In [2]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score

if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('custom_model1.h5')

        # Making predictions
        y_pred = model.predict(test_generator)
        y_pred = np.argmax(y_pred, axis=1)

        # The true labels can be accessed via the classes attribute of the generator
        # y_true = test_generator.classes

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0)
        y_true = np.argmax(y_true, axis=1)

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)

        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')

else:
    model = load_model('custom_model1.h5')

    # Making predictions
    y_pred = model.predict(test_generator)
    y_pred = np.argmax(y_pred, axis=1)

    # The true labels can be accessed via the classes attribute of the generator
    # y_true = test_generator.classes

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0)
    y_true = np.argmax(y_true, axis=1)

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu


KeyboardInterrupt



### Custom model best

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score

if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('custom_model_4_blocks_best.h5')

        # Making predictions
        y_pred = model.predict(test_generator)
        y_pred = np.argmax(y_pred, axis=1)

        # The true labels can be accessed via the classes attribute of the generator
        # y_true = test_generator.classes

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0)
        y_true = np.argmax(y_true, axis=1)

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)
        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')

else:
    model = load_model('custom_model_4_blocks_best.h5')

    # Making predictions
    y_pred = model.predict(test_generator)
    y_pred = np.argmax(y_pred, axis=1)

    # The true labels can be accessed via the classes attribute of the generator
    # y_true = test_generator.classes

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0)
    y_true = np.argmax(y_true, axis=1)

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')

### Resnet 50

In [ ]:
if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('resnet50.h5')

        # Making predictions
        y_pred = model.predict(test_generator)
        y_pred = np.argmax(y_pred, axis=1)

        # The true labels can be accessed via the classes attribute of the generator
        # y_true = test_generator.classes

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0)
        y_true = np.argmax(y_true, axis=1)

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)

        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')
else:
    model = load_model('resnet50.h5')

    # Making predictions
    y_pred = model.predict(test_generator)
    y_pred = np.argmax(y_pred, axis=1)

    # The true labels can be accessed via the classes attribute of the generator
    # y_true = test_generator.classes

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0)
    y_true = np.argmax(y_true, axis=1)

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')

2023-05-28 20:08:54.683903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-28 20:08:54.684100: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-28 20:08:54.684880: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

79/79 [==============================] - 17s 142ms/step
F1 Score: 0.18993428852847458


## Attractive

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

import tensorflow as tf
import pandas as pd
import numpy as np
attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Attractive']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

features = ['Attractive']
for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')


batch_size = 64
img_height = 128
img_width = 128

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)


Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [4]:
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('best_model_attractive.h5')

        # Making predictions
        y_pred_proba = model.predict(test_generator).flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()
        # y_true = np.argmax(y_true, axis=1)

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='weighted')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)

        auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
        print('AUC:', auc)

        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')
else:
    model = load_model('best_model_attractive.h5')

    # Making predictions
    y_pred_proba = model.predict(test_generator).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()
    # y_true = np.argmax(y_true, axis=1)

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
    print('AUC:', auc)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')

2023-06-01 20:22:24.158081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:22:24.158350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:22:24.160142: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

79/79 [==============================] - 25s 150ms/step
F1 Score: 0.8005226712633323
AUC: 0.8917647637254604


2023-06-01 20:22:58.465762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:22:58.466041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:22:58.467222: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

Test loss for the model : 0.4126
Test accuracy for the model: 0.8014


In [5]:
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('resnet_attractive.h5')

        # Making predictions
        y_pred_proba = model.predict(test_generator).flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)

        # The true labels can be accessed via the classes attribute of the generator
        # y_true = test_generator.classes

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()
        # y_true = np.argmax(y_true, axis=1)

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)

        auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
        print('AUC:', auc)

        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')
else:
    model = load_model('resnet_attractive.h5')

    # Making predictions
    y_pred_proba = model.predict(test_generator).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)

    # The true labels can be accessed via the classes attribute of the generator
    # y_true = test_generator.classes

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()
    # y_true = np.argmax(y_true, axis=1)

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
    print('AUC:', auc)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')


2023-06-01 20:23:31.749114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:23:31.749349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:23:31.750517: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

79/79 [==============================] - 19s 168ms/step
F1 Score: 0.6976696925459894
AUC: 0.8445248300347642


2023-06-01 20:23:56.092933: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:23:56.093172: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:23:56.094353: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

Test loss for the model : 0.5964
Test accuracy for the model: 0.7074


## Bangs

In [6]:
attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Bangs']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

features = ['Bangs']
for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')


batch_size = 64
img_height = 128
img_width = 128

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)


Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [7]:
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('best_model_bangs.h5')

        # Making predictions
        y_pred_proba = model.predict(test_generator).flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)

        # The true labels can be accessed via the classes attribute of the generator
        # y_true = test_generator.classes

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)

        auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
        print('AUC:', auc)

        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')
else:
    model = load_model('best_model_bangs.h5')

    # Making predictions
    y_pred_proba = model.predict(test_generator).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)

    # The true labels can be accessed via the classes attribute of the generator
    # y_true = test_generator.classes

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
    print('AUC:', auc)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')

2023-06-01 20:24:34.507009: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:24:34.507384: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:24:34.509468: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

79/79 [==============================] - 15s 168ms/step
F1 Score: 0.9168627450980393
AUC: 0.9867186867722846


2023-06-01 20:24:58.297480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:24:58.297763: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:24:58.299026: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

Test loss for the model : 0.1053
Test accuracy for the model: 0.9576


In [8]:
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

if use_multi_gpu:
    with strategy.scope():
    # Load the pre-trained model from the h5 file
        model = load_model('resnet_bangs.h5')

        # Making predictions
        y_pred_proba = model.predict(test_generator).flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)

        # The true labels can be accessed via the classes attribute of the generator
        # y_true = test_generator.classes

        # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
        y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()

        # Compute F1 score
        f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
        print('F1 Score:', f1)

        auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
        print('AUC:', auc)

        loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
        print(f'Test loss for the model : {loss:.4f}')
        print(f'Test accuracy for the model: {accuracy:.4f}')
else:
    model = load_model('resnet_bangs.h5')

    # Making predictions
    y_pred_proba = model.predict(test_generator).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)

    # The true labels can be accessed via the classes attribute of the generator
    # y_true = test_generator.classes

    # Since class_mode was set to 'raw' in the test_generator, classes contain the raw labels
    y_true = np.concatenate([test_generator[i][1] for i in range(len(test_generator))], axis=0).flatten()

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' for unweighted mean, 'weighted' for weighted mean
    print('F1 Score:', f1)

    auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # 'ovr' for One-vs-Rest, 'ovo' for One-vs-One
    print('AUC:', auc)

    loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
    print(f'Test loss for the model : {loss:.4f}')
    print(f'Test accuracy for the model: {accuracy:.4f}')

2023-06-01 20:25:29.005866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:25:29.006059: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:25:29.006913: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

79/79 [==============================] - 21s 186ms/step
F1 Score: 0.8604089937617805
AUC: 0.9630967741935483


2023-06-01 20:25:59.218881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:25:59.219282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-01 20:25:59.221199: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

Test loss for the model : 0.1738
Test accuracy for the model: 0.9292
